<a href="https://colab.research.google.com/github/AdemilsonMiguel/ProjetoVsCode/blob/main/Bandas_Bollinger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.graph_objects as go

In [4]:
# Pegar a ação 
acao = yf.Ticker('PETR4.SA')
data = acao.history(period='1y')
df = data[['Close']]

In [5]:
df

,Close
Date,
2021-07-22,18.537863
2021-07-23,18.427601
2021-07-26,18.930672
2021-07-27,18.710150
2021-07-28,19.096066
...,...
2022-07-18,28.600000
2022-07-19,29.180000
2022-07-20,29.170000


In [6]:
# Calcular media movel
mm = df.rolling(window=20).mean()
mm

,Close
Date,
2021-07-22,NaN
2021-07-23,NaN
2021-07-26,NaN
2021-07-27,NaN
2021-07-28,NaN
...,...
2022-07-18,27.9500
2022-07-19,28.0555
2022-07-20,28.1645


In [7]:
# SD
dp = df.rolling(window=20).std()
dp

,Close
Date,
2021-07-22,NaN
2021-07-23,NaN
2021-07-26,NaN
2021-07-27,NaN
2021-07-28,NaN
...,...
2022-07-18,0.757183
2022-07-19,0.774905
2022-07-20,0.770451


In [8]:
# Calculo banda sup e inf
sup_band = mm + 2 * dp
inf_band = mm - 2 * dp

In [9]:
# Alterar nome das colunas para sup e inf
sup_band = sup_band.rename(columns = {'Close': 'Superior'})
inf_band = inf_band.rename(columns = {'Close': 'Inferior'})

In [10]:
# Unir colunas
bandas_bollinger = df.join(sup_band).join(inf_band)

In [11]:
bandas_bollinger

,Close,Superior,Inferior
Date,,,
2021-07-22,18.537863,NaN,NaN
2021-07-23,18.427601,NaN,NaN
2021-07-26,18.930672,NaN,NaN
2021-07-27,18.710150,NaN,NaN
2021-07-28,19.096066,NaN,NaN
...,...,...,...
2022-07-18,28.600000,29.464366,26.435634
2022-07-19,29.180000,29.605311,26.505689
2022-07-20,29.170000,29.705401,26.623599


In [12]:
# Retirar nulos
bandas_bollinger.dropna(inplace=True)

In [13]:
# Calculo pontos de compra e venda
compra = bandas_bollinger[bandas_bollinger['Close'] <= bandas_bollinger['Inferior']]
venda = bandas_bollinger[bandas_bollinger['Close'] <= bandas_bollinger['Superior']]

In [14]:
pio.templates.default = 'plotly_dark'

fig = go.Figure()
fig.add_trace(go.Scatter(x=inf_band.index,
                        y=inf_band['Inferior'],
                        name='Banda_Inferior',
                        line_color='rgba(173,204,255,0.2)'
                        ))

fig.add_trace(go.Scatter(x=sup_band.index,
                        y=sup_band['Superior'],
                        name='Banda_Superior',
                        fill='tonexty',
                        fillcolor='rgba(173,204,255,0.2)',
                        line_color='rgba(173,204,255,0.2)'
                        ))

fig.add_trace(go.Scatter(x=df.index,
                        y=df['Close'],
                        name='Preco Fechamento',
                        line_color='#636EFA'
                        ))

fig.add_trace(go.Scatter(x=mm.index,
                        y=mm['Close'],
                        name='Media Movel',
                        line_color='#FECB52'
                        ))

fig.add_trace(go.Scatter(x=compra.index,
                        y=compra['Close'],
                        name='Compra',
                        mode='markers',
                        marker=dict(
                            color='#00CC96',
                            size=8,
                            )
                        
                        ))
fig.add_trace(go.Scatter(x=venda.index,
                        y=venda['Close'],
                        name='Venda',
                        mode='markers',
                        marker=dict(
                            color='#EF553B',
                            size=8,
                            )
                        
                        ))
fig.show()